In [2]:
import openai
import json
import time

data processor

In [6]:
## for test_s.tsv
def processor1(path):
    lines=[]
    with open(path,'r',encoding='utf-8')as f:
        for _,line in enumerate(f):
            if _==0:
                continue
            text,question,label = line.split('\t')
            label=label.strip()
            text = "\n".join([text,question])
            lines.append((text,label))
    return lines

In [3]:
def processor2(path,flags):
    lines=[]
    with open(path,'r',encoding='utf-8')as f:
        for _,line in enumerate(f):
            if _==0:
                continue
            text,question,label = line.split('\t')
            label=label.strip()
            text_list=text.split(" ")
            sentence=[]
            sentences=""
            for word in text_list:
                if word not in flags:
                    sentence.append(word)
                else:
                    sentences+=(" ".join(sentence)+"\n")
                    sentence=[word]
            sentences+=(" ".join(sentence))
            text = "\n".join([sentences,question])
            lines.append((text,label))
    return lines

class for chatgpt

In [7]:
class ChatGPT4CSC(object):
    def __init__(self,key_file,message_file=None):
        self.key_file=key_file
        self.openai_key=None
        self.messages={"role": "system", "content": "you are an assistant to judge if a sentence is true or false"}
        self.message_file=message_file##directory to store the chat messages if neccesary
    
    def get_api_key(self):
        with open(self.key_file, 'r', encoding='utf-8') as f:
            self.openai_key = json.load(f)[0]["api_key"]
            openai.api_key=self.openai_key
    
    def gptCorrect(self,text):
        result=openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=[
                        {"role": "system", "content": "Please answer the question with a 'true' or 'false' without details about justifications"},
                        {"role": "user", "content": text},
                    ]
                )
        return result.get("choices")[0].get("message").get("content") ## the response of chatgpt

In [8]:

chat=ChatGPT4CSC(key_file="../envs/openai_key")
chat.get_api_key()

do request for test_s.csv

In [8]:
lines=processor1("../data/test_s.tsv")
print(lines[0][0])
print(lines[0][1])
print(len(lines))

S5: John was analyzing some of the results from his sunscreen study. He found that a lower SPF protects the skin Longer
Is S5 true or false?
1
1000


In [ ]:
result=[]
out_file="test_s_result.txt"
with open(out_file,'w',encoding='utf-8') as f:
    for _,(text,label) in enumerate(lines):
        while(1):
            try:
                prediction=chat.gptCorrect(text)
            except:
                time.sleep(3)
                continue
            result.append((text,label,prediction))
            message="\t".join([text.strip(),label.strip(),prediction.strip()])+'\n'
            print(message)
            f.write(message)
            break

do request for test_r.csv

In [4]:
flags=[]
for i in range(2,20):
    flags.append("S{}:".format(i))

In [5]:
lines=processor2("../data/test_r.tsv",flags)
print(lines[0][0])
print(lines[0][1])
print(len(lines))

S1: John was analyzing some of the results from his sunscreen study. He found that a lower SPF protects the skin Longer
S2: S1 is a false statement.
S3: S2 is a false statement.
S4: S3 is a false statement.
S5: S4 is a false statement.
S6: S5 is a false statement.
S7: S6 is a true statement.
S8: S7 is a true statement.
S9: S8 is a true statement.
Is S9 true or false?
0
1000


In [ ]:
result=[]
out_file="test_r_result.txt"
with open(out_file,'w',encoding='utf-8') as f:
    for _,(text,label) in enumerate(lines):
        while(1):
            try:
                prediction=chat.gptCorrect(text)
            except:
                time.sleep(3)
                continue
            result.append((text,label,prediction))
            message="\t".join([label.strip(),prediction.strip()])+'\n'
            print(message)
            f.write(message)
            break